In [10]:
import numpy as np
import time
import os, shlex
from subprocess import Popen, PIPE, STDOUT, TimeoutExpired

In [11]:
class Submit(object):
    '''
        1) arg to loop over 
        2) maximum number of nodes to use at once
        3) maximum number of jobs to submit
    '''
    def name(self):
        return self.__class__.__name__

    def execute(self, *args, **kwargs):
        eval('self.{name}(*args, **kwargs)'.format(name = self.name().lower()))

    def submit(self, loop_over=[], max_node_in_use=10, max_job=3):
        _start = 0
        self.max_node_in_use = max_node_in_use
        self.loop_over = loop_over

        index = _start
        while (index < len(self.loop_over) and index < max_job):
            if (self.working_job() < self.max_node_in_use):
                self._submit_single(loop_index=index,
                                    loop_item=self.loop_over[index])

                time.sleep(0.1)
                index += 1

    # check my jobs
    def working_job(self):
        cmd='squeue --user={user} | wc -l'.format(user = 'zhut')
        jobNum=int(os.popen(cmd).read())
        return  jobNum -1

    def _submit_single(self, loop_index, loop_item):
        JOBNAME='paxwf{:03}'.format(loop_index)
        RUNNAME=str(loop_item)

        cmd = ('/home/zhut/submission/submit.py'
               ' --cpu 1'
               ' --cpu-memmory 1200'
               ' --job {JOBNAME}'
               ' --command "python {cwd}/pax_wfsim_job.py {RUNNAME}"')

        cmd = cmd.format(JOBNAME=JOBNAME, RUNNAME=RUNNAME, cwd=os.getcwd())
        cmd_args = shlex.split(cmd)
        
        with Popen(cmd, shell=True, stderr=STDOUT, stdout=PIPE) as proc:
            try:
                stdout, stderr = proc.communicate(
                    timeout=1800)
                #print(stdout.decode('utf-8'))
            except TimeoutExpired:
                print('Subprocess timed out at 1800s')
                proc.kill()
                stdout, stderr = proc.communicate(
                    timeout=1800)
                #print(stdout.decode('utf-8'))

In [12]:
p = Submit()
p.execute(loop_over=np.arange(20, 22), max_node_in_use=64, max_job=1)